In [21]:
# Imports
import os
import json
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery, QueryType, QueryCaptionType, QueryAnswerType, VectorFilterMode 
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

# Environment setup
load_dotenv()
deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
language="python"
service_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_KEY") 
model = "ada002"

#Initialize AzureOpenAI client
client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_KEY'],  
  api_version = "2023-12-01-preview"
  )

messages=[]

# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
# ** Important!** Use with AZURE data only
def get_from_RAG(query: str) :    
    search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
    vector_query = VectorizedQuery(vector= generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector")
    results = search_client.search(  
        search_text=query,  
        vector_queries=[vector_query],
        select=["title", "content", "category"],
        query_type=QueryType.SEMANTIC, 
        semantic_configuration_name='my-semantic-config', 
        query_caption=QueryCaptionType.EXTRACTIVE, 
        query_answer=QueryAnswerType.EXTRACTIVE,
        top=3
    )
    semantic_answers = results.get_answers()    
    for answer in semantic_answers:
        print(f"Semantic Answer Score: {answer.score}")
        print(f"Semantic Answer: {answer.text}\n")            
        
    rag_results=[]
    for result in results:                    
        print(f"Title: {result['title']}")
        print(f"Reranker Score: {result['@search.reranker_score']}")
        print(f"Content: {result['content']}")
        print(f"Category: {result['category']}")
        captions = result["@search.captions"]
        if captions:
            caption = captions[0]
            print(f"Caption: {caption.text}\n")
                    
                
    return rag_results

result= get_from_RAG("ExpressRoute ")
print(result)

In [15]:
# Pure Vector Search
query = "SE-01 Soldering"  
  
search_client = SearchClient(service_endpoint, index_name, credential=AzureKeyCredential(key))
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "categories"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['categories']}\n") 

Title: SE-01
Score: 0.80322015
Content: Is a multi operation machine that can handle all the operations of a standard machine shop.\nIt is a combination of cutting, welding and blowing machine. It can be used for cutting, welding and blowing operations.\nSupports vertical,horizontal and round cutting and soldering.
Category: Blowing, Soldering, Cutting

Title: SE-01: Horizontal Soldering Library
Score: 0.80224997
Content: The library package 'machine_solder_h' contains functions related to material soldering.\nHere's the list of available functions:\n# Weld the material horizontally from the point (beginX,beginY) to the point (endX,endY)\nsold_H(beginX, beginY, endX, endY)\n# Weld the material horizontally from the point (beginX,beginY) to the point (endX,endY) in a zigzag pattern\nH_ZZ_Sold(beginX, beginY, endX, endY)
Category: Soldering

Title: SE-01: Vertical Soldering Library
Score: 0.7903209
Content: The library package 'machine_solder_v' contains functions related to material sol

In [17]:
# Exhaustive Vector Search
query = "SE-01 Soldering"  
  
search_client = SearchClient(service_endpoint, index_name, credential=AzureKeyCredential(key))
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector", exhaustive=True)
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "categories"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['categories']}\n") 

Title: SE-01
Score: 0.8032199
Content: Is a multi operation machine that can handle all the operations of a standard machine shop.\nIt is a combination of cutting, welding and blowing machine. It can be used for cutting, welding and blowing operations.\nSupports vertical,horizontal and round cutting and soldering.
Category: Blowing, Soldering, Cutting

Title: SE-01: Horizontal Soldering Library
Score: 0.80224997
Content: The library package 'machine_solder_h' contains functions related to material soldering.\nHere's the list of available functions:\n# Weld the material horizontally from the point (beginX,beginY) to the point (endX,endY)\nsold_H(beginX, beginY, endX, endY)\n# Weld the material horizontally from the point (beginX,beginY) to the point (endX,endY) in a zigzag pattern\nH_ZZ_Sold(beginX, beginY, endX, endY)
Category: Soldering

Title: SE-01: Vertical Soldering Library
Score: 0.7903207
Content: The library package 'machine_solder_v' contains functions related to material sold

In [18]:

# Cross field Vector Search
query = "SE-01 Soldering"  
  
search_client = SearchClient(service_endpoint, index_name, credential=AzureKeyCredential(key))
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector, titleVector")  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "categories"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['categories']}\n") 

Title: SE-01: Horizontal Soldering Library
Score: 0.03306011110544205
Content: The library package 'machine_solder_h' contains functions related to material soldering.\nHere's the list of available functions:\n# Weld the material horizontally from the point (beginX,beginY) to the point (endX,endY)\nsold_H(beginX, beginY, endX, endY)\n# Weld the material horizontally from the point (beginX,beginY) to the point (endX,endY) in a zigzag pattern\nH_ZZ_Sold(beginX, beginY, endX, endY)
Category: Soldering

Title: SE-01
Score: 0.03279569745063782
Content: Is a multi operation machine that can handle all the operations of a standard machine shop.\nIt is a combination of cutting, welding and blowing machine. It can be used for cutting, welding and blowing operations.\nSupports vertical,horizontal and round cutting and soldering.
Category: Blowing, Soldering, Cutting

Title: SE-01: Vertical Soldering Library
Score: 0.032522473484277725
Content: The library package 'machine_solder_v' contains func

In [19]:
# Multi Vector Search
query = "SE-01 Soldering"  
  
search_client = SearchClient(service_endpoint, index_name, credential=AzureKeyCredential(key))
vector_query_1 = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="titleVector")
vector_query_2 = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query_1, vector_query_2],
    select=["title", "content", "categories"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['categories']}\n") 

Title: SE-01: Horizontal Soldering Library
Score: 0.03306011110544205
Content: The library package 'machine_solder_h' contains functions related to material soldering.\nHere's the list of available functions:\n# Weld the material horizontally from the point (beginX,beginY) to the point (endX,endY)\nsold_H(beginX, beginY, endX, endY)\n# Weld the material horizontally from the point (beginX,beginY) to the point (endX,endY) in a zigzag pattern\nH_ZZ_Sold(beginX, beginY, endX, endY)
Category: Soldering

Title: SE-01
Score: 0.03279569745063782
Content: Is a multi operation machine that can handle all the operations of a standard machine shop.\nIt is a combination of cutting, welding and blowing machine. It can be used for cutting, welding and blowing operations.\nSupports vertical,horizontal and round cutting and soldering.
Category: Blowing, Soldering, Cutting

Title: SE-01: Vertical Soldering Library
Score: 0.032522473484277725
Content: The library package 'machine_solder_v' contains func

In [27]:
# Search with filter
query = "SE-01 Soldering"  
  
search_client = SearchClient(service_endpoint, index_name, credential=AzureKeyCredential(key))
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector")  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter="categories eq 'Soldering'",
    select=["title", "content", "categories"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['categories']}\n") 

Title: SE-01: Horizontal Soldering Library
Score: 0.80224997
Content: The library package 'machine_solder_h' contains functions related to material soldering.\nHere's the list of available functions:\n# Weld the material horizontally from the point (beginX,beginY) to the point (endX,endY)\nsold_H(beginX, beginY, endX, endY)\n# Weld the material horizontally from the point (beginX,beginY) to the point (endX,endY) in a zigzag pattern\nH_ZZ_Sold(beginX, beginY, endX, endY)
Category: Soldering

Title: SE-01: Vertical Soldering Library
Score: 0.7903207
Content: The library package 'machine_solder_v' contains functions related to material soldering.\nHere's the list of available functions:\n# Solder the material vertically from the point (beginX,beginY) to the point (endX,endY)\nvSold(beginX, beginY, endX, endY)\n# Solder the material vertically from the point (beginX,beginY) to the point (endX,endY) in a zigzag pattern
v_S_ZZCut(beginX, beginY, endX, endY)
Category: Soldering

Title: SE-01:

In [33]:
# Hybrid Search
query = "SE-01 Soldering"  
  
search_client = SearchClient(service_endpoint, index_name, credential=AzureKeyCredential(key))
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector")  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    top=3,
    select=["title", "content", "categories", "doctype"],
    query_type=QueryType.SEMANTIC, 
    semantic_configuration_name='my-semantic-config', 
    query_caption=QueryCaptionType.EXTRACTIVE, 
    query_answer=QueryAnswerType.EXTRACTIVE    
)  
  
semantic_answers = results.get_answers()
for answer in semantic_answers:
    print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['categories']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        print(f"Caption: {caption.text}\n")
            

Semantic Answer: SE-01. Blowing, Soldering, Cutting. Is a multi operation machine that can handle all the operations of a standard machine shop.\nIt is a combination of cutting, welding and blowing machine. It can be used for cutting, welding and blowing operations.\nSupports vertical,horizontal and round cutting and soldering..
Semantic Answer Score: 0.96728515625

Title: SE-01
Reranker Score: 3.018075942993164
Content: Is a multi operation machine that can handle all the operations of a standard machine shop.\nIt is a combination of cutting, welding and blowing machine. It can be used for cutting, welding and blowing operations.\nSupports vertical,horizontal and round cutting and soldering.
Category: Blowing, Soldering, Cutting
Caption: SE-01. Blowing, Soldering, Cutting. Is a multi operation machine that can handle all the operations of a standard machine shop.\nIt is a combination of cutting, welding and blowing machine. It can be used for cutting, welding and blowing operations.\n